## Example - testing kinetic advection using a wave problem 

This example tests the kinetic advection operators by building a toy wave problem from the Vlasov advection terms for f0 and f1

This is a v2.0.0 rewrite of the notebook that corresponds to Section 5.2.1. of the ReMKiT1D code paper.

In [ ]:
import numpy as np
import holoviews as hv
import matplotlib.pyplot as plt

import RMK_support as rmk
import RMK_support.common_models as cm

### Context initialization

In [ ]:
rk = rmk.RMKContext()
rk.IOContext=rmk.IOContext(HDF5Dir="./RMKOutput/RMK_kin_adv_test/" )
rk.mpiContext = rmk.MPIContext(4)

### Grid setup

In [ ]:
xGrid = 0.1*np.ones(128) 
L = sum(xGrid)
vGrid = np.logspace(-2,1,80) #In normalized velocity - default normalization is thermal velocity sqrt(m_e * k * T_e/2)
lMax = 1
rk.grid = rmk.Grid(xGrid,vGrid,lMax,interpretXGridAsWidths=True,isPeriodic=True)

### Variables

In [ ]:
n = np.exp(-(rk.grid.xGrid-np.mean(rk.grid.xGrid))**2) # A Gaussian spatial profile

fInit = np.zeros([rk.grid.numX,rk.grid.numH,rk.grid.numV])
for i in range(rk.grid.numV):
    fInit[:,rk.grid.getH(0)-1,i] = n 

f = rmk.Variable("f",rk.grid,data=fInit,isDistribution=True)
rk.variables.add(f)

### Adding the advection model

The spatial advection model is implemented in common_models.py and is just used directly here. For implementation details see the corresponding function.

In [ ]:
rk.models.add(cm.kinAdvX(f,rk.grid))

### Integrator and timestep options

Simple single step backwards Euler integration

In [ ]:
integrator = rmk.BDEIntegrator("BDE",absTol=10.0,convergenceVars=[f])
integrationStep = rmk.IntegrationStep("BE",integrator)
integrationStep.add(rk.models) 
rk.integrationScheme = rmk.IntegrationScheme(dt=0.01,steps=integrationStep) 
rk.integrationScheme.setFixedNumTimesteps(4000,40) 

#### Generate a LaTeX summary of the ReMKiT1D run 

In [ ]:
rk.generatePDF("Kinetic Advection Wave test")

### Create config file

In [ ]:
rk.writeConfigFile()

### Data analysis


#### Loading data

In [ ]:
loadedData = rk.loadSimulation()
dataset = loadedData.dataset

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=150,dpi=150)

### Compare to analytic travelling wave solution

In [ ]:
wave_speeds=rk.grid.vGrid/np.sqrt(3)
times = dataset.coords['t'].data
f0_analytic=np.zeros((len(times),rk.grid.numX,rk.grid.numV))

for i in range(len(times)):
    for k,c in enumerate(wave_speeds):
        leftPositionMod = (rk.grid.xGrid-c*times[i]) % L
        leftPosition = np.where(leftPositionMod > 0,leftPositionMod,leftPositionMod+L)
        rightPosition = (rk.grid.xGrid+c*times[i]) % L
        f0_analytic[i,:,k] = 0.5*(np.exp(-(leftPosition-np.mean(rk.grid.xGrid))**2) + np.exp(-(rightPosition-np.mean(rk.grid.xGrid))**2))


In [ ]:
dataName = 'f'
vInd = 40 # Lower velocities will be less diffusive
curveDict = {t: hv.Scatter(dataset[dataName][{"t":t,"h":0,"v":vInd}]).opts(marker="o",color="r",s=6.0)*hv.Curve((rk.grid.xGrid,f0_analytic[t,:,vInd])) for t in range(len(times))}
kdims = [hv.Dimension(('t', 'Time'),unit=dataset.coords["t"].attrs["units"], default=0)]
hv.HoloMap(curveDict,kdims=kdims)

### Reproducing the plot in the paper

In [ ]:
diff = np.abs(f0_analytic - dataset['f'][:,:,0,:])

In [ ]:
plotVPoints = [0,19,39,59]

In [ ]:
relativeErrorPlot=hv.Overlay([hv.Curve(diff.reduce(np.max,'x')[:,v],label="$v={:.2f}$".format(rk.grid.vGrid[v])+'$ v_{th}$').opts(ylabel='$\Delta f_0$',linestyle='--',fontscale=1.5, fig_size=150,linewidth=2.0,logy=True,ylim=(1e-7,1)) for v in plotVPoints]).opts(legend_position='top',legend_cols=2)

In [ ]:
hv.output(fig='pdf')
hv.save(relativeErrorPlot, 'advectionKinTestRelErr.pdf', dpi=144)